In [3]:
# Standard imports
import numpy as np
import pandas as pd

# For web scraping
import requests
from bs4 import BeautifulSoup

# For performing regex operations
import re

# For adding delays so that we don't spam requests
import time

In [23]:
#Elastic search query used: filtering on before 2013
'''$ curl --location --request POST 'https://planningdata.london.gov.uk/api-guest/applications/_search' --header 'X-API-AllowRequest: be2rmRnt&' --header 'Content-Type: application/json' --data-raw '{"query": {"bool": {"must": [{"range": {"valid_date": {"lt": "01/01/2013"}}}]}}, "_source": ["lpa_name", "lpa_app_no", "last_updated", "valid_date", "decision_date", "id", "application_type"]}' > old.json'''

'$ curl --location --request POST \'https://planningdata.london.gov.uk/api-guest/applications/_search\' --header \'X-API-AllowRequest: be2rmRnt&\' --header \'Content-Type: application/json\' --data-raw \'{"query": {"bool": {"must": [{"range": {"valid_date": {"lt": "01/01/2013"}}}]}}, "_source": ["lpa_name", "lpa_app_no", "last_updated", "valid_date", "decision_date", "id", "application_type"]}\' > old.json'

In [29]:
before2013_raw = pd.read_json('old.json')
before2013_raw

,took,timed_out,_shards,hits
total,1289,False,1.0,"{'value': 8231, 'relation': 'eq'}"
successful,1289,False,1.0,NaN
skipped,1289,False,0.0,NaN
failed,1289,False,0.0,NaN
max_score,1289,False,NaN,1.0
hits,1289,False,NaN,"[{'_index': 'applications', '_type': '_doc', '..."


In [30]:
before2013_raw = pd.json_normalize(before2013_raw.loc['hits', 'hits'])
before2013_raw.sort_values('_source.valid_date')
before2013_raw.groupby('_source.lpa_name').count()['_index']

_source.lpa_name
Barnet             869
Bexley               4
Bromley            262
Camden              35
City of London     194
Croydon              5
Ealing            6326
Enfield             20
Hackney             11
Havering            53
Hillingdon          31
LLDC                 1
Lambeth            185
Merton               9
Redbridge            8
Richmond           167
Wandsworth          23
Westminster         28
Name: _index, dtype: int64

Let's look at the ditribution for Hackney (as it was majorly gentrified -> important for the study)

In [35]:
#elastic search query for applications before 2021
'''$ curl --location --request POST 'https://planningdata.london.gov.uk/api-guest/applications/_search' --header 'X-API-AllowRequest: be2rmRnt&' --header 'Content-Type: application/json' --data-raw '{"query": {"bool": {"must": [{"term":{"lpa_name.raw": "Hackney"}}, {"range": {"valid_date": {"lt": "01/01/2021"}}}]}}, "_source": ["lpa_name", "lpa_app_no", "last_updated", "valid_date", "decision_date", "id", "application_type"], "size" : 10000}' > old_Hackney.json'''

'$ curl --location --request POST \'https://planningdata.london.gov.uk/api-guest/applications/_search\' --header \'X-API-AllowRequest: be2rmRnt&\' --header \'Content-Type: application/json\' --data-raw \'{"query": {"bool": {"must": [{"term":{"lpa_name.raw": "Hackney"}}, {"range": {"valid_date": {"lt": "01/01/2021"}}}]}}, "_source": ["lpa_name", "lpa_app_no", "last_updated", "valid_date", "decision_date", "id", "application_type"], "size" : 10000}\' > old_Hackney.json'

In [34]:
Hackney_all = pd.read_json('old_Hackney.json')
Hackney_all

,took,timed_out,_shards,hits
total,323,False,1.0,"{'value': 969, 'relation': 'eq'}"
successful,323,False,1.0,NaN
skipped,323,False,0.0,NaN
failed,323,False,0.0,NaN
max_score,323,False,NaN,5.152546
hits,323,False,NaN,"[{'_index': 'applications', '_type': '_doc', '..."


In [59]:
#only 969 applications before 2021 for Hackney

# Exploring Recent Applications for Hackney

In [7]:
#load json for Hackney
test_all = pd.read_json('test_all.json')
my_test_all=test_all.loc['hits', 'hits']

In [8]:
test7 = pd.read_json('test7.json')

In [9]:
my_test=test7.loc['hits', 'hits']

In [221]:
my_test[0]

{'_index': 'applications',
 '_type': '_doc',
 '_id': 'Hackney-2022_2840',
 '_score': 5.1420603,
 '_source': {'lpa_name': 'Hackney',
  'decision_date': None,
  'last_updated': '2023-02-22T20:00:15.896Z',
  'valid_date': '07/02/2023',
  'lpa_app_no': '2022/2840',
  'application_type': 'Householder',
  'centroid': {'lon': -7.55716, 'lat': 49.766807},
  'postcode': 'N16 6TS',
  'application_details': {'total_gia_gained': 5},
  'id': 'Hackney-2022_2840',
  'application_type_full': 'Householder planning permission'}}

In [12]:
#How can we convert json into a dataframe?
'''my_test[0].keys()
#Tryiing to construct a dataframe with dictionary keys as column names
#my_test
for key in my_test[0]:
    print(key)
    #df_new['key']=app_dict[key]
    #print(df_new)''';


# Let's look at all the columns?

In [41]:
all_columns=pd.read_json('test1.json')
all_columns

,took,timed_out,_shards,hits
total,1555,False,1.0,"{'value': 4896, 'relation': 'eq'}"
successful,1555,False,1.0,NaN
skipped,1555,False,0.0,NaN
failed,1555,False,0.0,NaN
max_score,1555,False,NaN,4.852367
hits,1555,False,NaN,"[{'_index': 'applications', '_type': '_doc', '..."


In [43]:
len(all_columns.loc['hits', 'hits'])

4896

In [44]:
all_columns2 = pd.json_normalize(all_columns.loc['hits', 'hits'])
all_columns2.shape

(4896, 227)

In [45]:
(all_columns2.isna().sum().sort_values()*100/all_columns2.shape[0])[(all_columns2.isna().sum().sort_values()*100/all_columns2.shape[0])<50]

_index                                              0.000000
_source.bo_system                                   0.000000
_source.last_synced                                 0.000000
_source.last_updated_by                             0.000000
_source.lpa_name                                    0.000000
_source.centroid.lon                                0.000000
_source.centroid.lat                                0.000000
_source.application_type                            0.000000
_source.valid_date                                  0.000000
_source.lpa_app_no                                  0.000000
_source.id                                          0.000000
_score                                              0.000000
_id                                                 0.000000
_type                                               0.000000
_source.last_updated                                0.000000
_source.pp_id                                       4.248366
_source.application_type

In [46]:
(all_columns2.isna().sum().sort_values()*100/all_columns2.shape[0])[-50::];

# Let's look at data before 2009

In [47]:
all_old_js=pd.read_json('test_old.json')
all_old_js

,took,timed_out,_shards,hits
total,582,False,1.0,"{'value': 2709, 'relation': 'eq'}"
successful,582,False,1.0,NaN
skipped,582,False,0.0,NaN
failed,582,False,0.0,NaN
max_score,582,False,NaN,1.0
hits,582,False,NaN,"[{'_index': 'applications', '_type': '_doc', '..."


In [48]:

all_old_raw = pd.json_normalize(all_old_js.loc['hits', 'hits'])
all_old_raw.sort_values('_source.valid_date')

,_index,_type,_id,_score,_source.lpa_name,_source.last_updated,_source.valid_date,_source.application_type,_source.id,_ignored,_source.application_details.3d_model
1366,applications,_doc,Redbridge-PL_WW0013_62_A,1.0,Redbridge,2021-05-07T00:01:41.700Z,01/01/1962,All Other,Redbridge-PL_WW0013_62_A,NaN,False
1378,applications,_doc,Redbridge-PL_0662_89,1.0,Redbridge,2021-05-01T00:02:22.838Z,01/01/1989,All Other,Redbridge-PL_0662_89,NaN,False
1811,applications,_doc,Havering-P1597.19,1.0,Havering,2021-11-17T04:11:57.505Z,01/01/2001,All Other,Havering-P1597.19,NaN,NaN
2429,applications,_doc,Ealing-P_2002_0257,1.0,Ealing,2022-11-21T09:01:20.840Z,01/02/2002,All Other,Ealing-P_2002_0257,NaN,NaN
2312,applications,_doc,Barnet-W09502C_05,1.0,Barnet,2022-08-29T20:01:44.577Z,01/02/2005,All Other,Barnet-W09502C_05,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
582,applications,_doc,Ealing-P_2003_4321,1.0,Ealing,2020-11-26T10:30:19.215Z,31/10/2003,All Other,Ealing-P_2003_4321,[polygon],NaN
1982,applications,_doc,Barnet-W16122_07,1.0,Barnet,2022-06-15T20:02:00.739Z,31/10/2007,All Other,Barnet-W16122_07,NaN,NaN
1362,applications,_doc,Barnet-C03312BK_07,1.0,Barnet,2021-05-06T20:01:46.960Z,31/10/2007,All Other,Barnet-C03312BK_07,NaN,NaN
1418,applications,_doc,Barnet-B_04096_08,1.0,Barnet,2021-04-22T20:02:18.024Z,31/10/2008,All Other,Barnet-B_04096_08,NaN,NaN


In [57]:
all_old_raw['_source.valid_date']=pd.to_datetime(all_old_raw['_source.valid_date'])
all_old_raw.info();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2709 entries, 0 to 2708
Data columns (total 11 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   _index                                2709 non-null   object        
 1   _type                                 2709 non-null   object        
 2   _id                                   2709 non-null   object        
 3   _score                                2709 non-null   float64       
 4   _source.lpa_name                      2709 non-null   object        
 5   _source.last_updated                  2709 non-null   object        
 6   _source.valid_date                    2709 non-null   datetime64[ns]
 7   _source.application_type              2709 non-null   object        
 8   _source.id                            2709 non-null   object        
 9   _ignored                              214 non-null    object        
 10  

In [50]:
all_old_raw.groupby('_source.lpa_name').count()['_index']

_source.lpa_name
Barnet             479
Bexley               1
Bromley            104
Camden              16
City of London      99
Croydon              3
Ealing            1697
Enfield             17
Hackney             10
Havering            36
Hillingdon          26
Lambeth             86
Merton               9
Redbridge            7
Richmond            88
Wandsworth          11
Westminster         20
Name: _index, dtype: int64

# Can we extract any useful information for Ealing?

In [51]:
ealing_js=pd.read_json('test_Ealing.json')
ealing_js

,took,timed_out,_shards,hits
total,28,False,1.0,"{'value': 141, 'relation': 'eq'}"
successful,28,False,1.0,NaN
skipped,28,False,0.0,NaN
failed,28,False,0.0,NaN
max_score,28,False,NaN,0.0
hits,28,False,NaN,"[{'_index': 'applications', '_type': '_doc', '..."


In [52]:
ealing_raw = pd.json_normalize(ealing_js.loc['hits', 'hits'])
#.sort_values('_source.valid_date')

In [55]:
ealing_raw['_source.valid_date']=pd.to_datetime(ealing_raw['_source.valid_date'])

In [56]:
ealing_raw.groupby(ealing_raw['_source.valid_date'].dt.month).count()['_source.lpa_name']

_source.valid_date
1      8
2      6
3     12
4     10
5     12
6      9
7     16
8     18
9     14
10    10
11    13
12    13
Name: _source.lpa_name, dtype: int64

In [153]:
df_test_10=pd.read_json('test_old2.json')
df_test_10

,took,timed_out,_shards,hits
total,1164,False,1.0,"{'value': 8122, 'relation': 'eq'}"
successful,1164,False,1.0,NaN
skipped,1164,False,0.0,NaN
failed,1164,False,0.0,NaN
max_score,1164,False,NaN,0.0
hits,1164,False,NaN,"[{'_index': 'applications', '_type': '_doc', '..."


In [154]:
df_test_10 = pd.json_normalize(df_test_10.loc['hits', 'hits'])
df_test_10.sort_values('_source.valid_date')
df_test_10.groupby('_source.lpa_name').count()['_index']

_source.lpa_name
Barnet             675
Bexley              11
Bromley            145
Camden              19
City of London      75
Croydon              7
Ealing            6341
Enfield             41
Hackney              1
Havering             6
Hillingdon           9
LLDC                34
Lambeth            108
Merton               1
Newham               1
OPDC                 1
Redbridge            1
Richmond            63
Tower Hamlets      560
Wandsworth           4
Westminster         19
Name: _index, dtype: int64

Can we extract data in other ways?

In [4]:
#test before finding out about the pd normalize function

'''from ukplanning import run

scraper = 'Hart'
scraper_dict = run.all_scraper_classes()
if not scraper in scraper_dict.keys():
    print 'Scraper class not yet implemented for s' % scraper
else:
    print 'Found scraper class for %s' % scraper
    scraper_obj = run.get_scraper(scraper, log_directory='logs')
    print 'Wait ...'
    scraped = scraper_obj.gather_ids()
    if scraped.get('result'):
        result = scraped['result']
        print "Found %d applications from %s to %s" % (len(result), scraped['from'], scraped['to'])
        app = result[0]
        updated = scraper_obj.update_application(app)
        if 'scrape_error' not in updated:
            print 'Updated first full record from %s' % scraper
            print app
            print 'OK'
        else:
            print 'Error during application update: %s' % updated['scrape_error']
    else:
        print 'Error when gathering applications: %s' % scraped['scrape_error']''';